# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation = 'relu', input_shape = (296,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation = 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 9ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [40]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051 entries, 0 to 1050
Data columns (total 36 columns):
MSSubClass       1051 non-null float64
LotFrontage      1051 non-null float64
LotArea          1051 non-null float64
OverallQual      1051 non-null float64
OverallCond      1051 non-null float64
YearBuilt        1051 non-null float64
YearRemodAdd     1051 non-null float64
MasVnrArea       1051 non-null float64
BsmtFinSF1       1051 non-null float64
BsmtFinSF2       1051 non-null float64
BsmtUnfSF        1051 non-null float64
TotalBsmtSF      1051 non-null float64
1stFlrSF         1051 non-null float64
2ndFlrSF         1051 non-null float64
LowQualFinSF     1051 non-null float64
GrLivArea        1051 non-null float64
BsmtFullBath     1051 non-null float64
BsmtHalfBath     1051 non-null float64
FullBath         1051 non-null float64
HalfBath         1051 non-null float64
BedroomAbvGr     1051 non-null float64
KitchenAbvGr     1051 non-null float64
TotRmsAbvGrd     1051 non-null floa

In [9]:
# Numeric column names
numeric_columns =  X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns= numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns = numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns = numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [11]:
# Train the model 
normalized_input_model.fit(X_train, y_train, batch_size= 32, epochs= 150, validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size = 32, epochs = 150, validation_data =(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.4708 - mse: 0.4708 - val_loss: 0.2245 - val_mse: 0.2245
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2541 - mse: 0.2541 - val_loss: 0.1593 - val_mse: 0.1593
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.1501 - val_mse: 0.1501
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1785 - mse: 0.1785 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1527 - mse: 0.1527 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1394 - mse: 0.1394 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1252 - mse: 0.1252 - val_loss: 0.1218 - val_mse: 0.1218
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1151 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1216 - val_mse: 0.1216
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1195 - val_mse: 0.1195
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1265 - val_mse: 0.1265
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1222 - val_mse: 0.1222
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0205 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1254 - val_mse: 0.1254
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1278 - val_mse: 0.1278
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1271 - val_mse: 0.1271
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 941us/step - loss: 0.0078 - mse: 0.0078


[0.007816166616976261, 0.007816166616976261]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1272 - mse: 0.1272


[0.1272287517786026, 0.1272287517786026]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [18]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val_pred, y_val))

28030.066106877366

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [21]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features,), kernel_initializer = 'he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.5158 - mse: 0.5158 - val_loss: 0.3630 - val_mse: 0.3630
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2770 - mse: 0.2770 - val_loss: 0.2492 - val_mse: 0.2492
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2182 - mse: 0.2182 - val_loss: 0.2066 - val_mse: 0.2066
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1907 - mse: 0.1907 - val_loss: 0.1847 - val_mse: 0.1847
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1707 - mse: 0.1707 - val_loss: 0.1705 - val_mse: 0.1705
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1556 - mse: 0.1556 - val_loss: 0.1661 - val_mse: 0.1661
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1483 - mse: 0.1483 - val_loss: 0.1536 - val_mse: 0.1536
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1383 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0303 - mse: 0.0303 - val_loss: 0.1492 - val_mse: 0.1492
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0295 - mse: 0.0295 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0295 - mse: 0.0295 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0292 - mse: 0.0292 - val_loss: 0.1491 - val_mse: 0.1491
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0284 - mse: 0.0284 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0279 - mse: 0.0279 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0274 - mse: 0.0274 - val_loss: 0.1511 - val_mse: 0.1511
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0273 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0141 - mse: 0.0141 - val_loss: 0.1582 - val_mse: 0.1582
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0143 - mse: 0.0143 - val_loss: 0.1582 - val_mse: 0.1582
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1566 - val_mse: 0.1566
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1580 - val_mse: 0.1580
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1570 - val_mse: 0.1570
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.1589 - val_mse: 0.1589
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 968us/step - loss: 0.0109 - mse: 0.0109


[0.010935629718005657, 0.010935629718005657]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1603 - mse: 0.1603


[0.16034869849681854, 0.16034869849681854]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [28]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation ='relu', input_shape = (n_features, ),kernel_initializer = 'lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: 0.4645 - mse: 0.4645 - val_loss: 0.1485 - val_mse: 0.1485
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2355 - mse: 0.2355 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1976 - mse: 0.1976 - val_loss: 0.1324 - val_mse: 0.1324
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1685 - mse: 0.1685 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1413 - mse: 0.1413 - val_loss: 0.0962 - val_mse: 0.0962
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1332 - mse: 0.1332 - val_loss: 0.0960 - val_mse: 0.0960
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1214 - mse: 0.1214 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1094 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1009 - val_mse: 0.1009
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1010 - val_mse: 0.1010
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1009 - val_mse: 0.1009
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1001 - val_mse: 0.1001
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0189 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1023 - val_mse: 0.1023
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1021 - val_mse: 0.1021
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1021 - val_mse: 0.1021
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1026 - val_mse: 0.1026
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1023 - val_mse: 0.1023
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071


[0.007082846015691757, 0.007082846015691757]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1036 - mse: 0.1036


[0.10364057868719101, 0.10364057868719101]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [32]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: 0.2799 - mse: 0.2799 - val_loss: 0.2073 - val_mse: 0.2073
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1522 - mse: 0.1522 - val_loss: 0.0925 - val_mse: 0.0925
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1138 - mse: 0.1138 - val_loss: 0.1101 - val_mse: 0.1101
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0983 - mse: 0.0983 - val_loss: 0.0905 - val_mse: 0.0905
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0770 - mse: 0.0770 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0571 - mse: 0.0571 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0568 - mse: 0.0568 - val_loss: 0.0983 - val_mse: 0.0983
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0524 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.0952 - val_mse: 0.0952
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1143 - val_mse: 0.1143
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1083 - val_mse: 0.1083
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.0933 - val_mse: 0.0933
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0070 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0906 - val_mse: 0.0906
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0927 - val_mse: 0.0927
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0880 - val_mse: 0.0880
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0962 - val_mse: 0.0962
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 957us/step - loss: 0.0030 - mse: 0.0030


[0.0029687429778277874, 0.0029687429778277874]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0959 - mse: 0.0959


[0.09587240219116211, 0.09587240219116211]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [36]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer = 'Adam', loss = 'mse', metrics = 'mse')

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.4222 - mse: 0.4222 - val_loss: 0.1262 - val_mse: 0.1262
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1563 - mse: 0.1563 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1182 - mse: 0.1182 - val_loss: 0.1374 - val_mse: 0.1374
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0784 - mse: 0.0784 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0609 - mse: 0.0609 - val_loss: 0.1352 - val_mse: 0.1352
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0535 - mse: 0.0535 - val_loss: 0.1326 - val_mse: 0.1326
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0384 - mse: 0.0384 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0292 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.1028 - val_mse: 0.1028
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1090 - val_mse: 0.1090
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0015 - mse: 0

33/33 [==============================] - 0s 2ms/step - loss: 4.4122e-04 - mse: 4.4122e-04 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 124/150
33/33 [==============================] - 0s 3ms/step - loss: 7.6073e-04 - mse: 7.6073e-04 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 9.7502e-04 - mse: 9.7502e-04 - val_loss: 0.1055 - val_mse: 0.1055
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0978 - val_mse: 0.0978
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0025 - mse: 0.0025 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 130/150
33/33 [==============================] - 0s 3

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [37]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 5.3500e-04 - mse: 5.3500e-04


[0.0005350022111088037, 0.0005350022111088037]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [38]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 1ms/step - loss: 0.0996 - mse: 0.0996


[0.09963156282901764, 0.09963156282901764]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [39]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1911 - mse: 0.1911


[0.19106006622314453, 0.19106006622314453]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [40]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

34349.18657845923

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.